# コードリーディング

TensorFLowによって2値分類を行うサンプルコードを載せました。今回はこれをベースにして進めます。


tf.kerasやtf.estimatorなどの高レベルAPIは使用していません。低レベルなところから見ていくことにします。



In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf

C:\Users\Kei\anaconda3\envs\py_env\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Kei\anaconda3\envs\py_env\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Kei\anaconda3\envs\py_env\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Kei\anaconda3\envs\py_env\lib\site-packages\tensorflow\python\frame

In [2]:
g = tf.Graph()

#計算グラフにノードを追加
with g.as_default():
    tf.set_random_seed(1)
    
    #プレースホルダを定義
    #次元の大きさが可変の時,Noneを指定
    tf_x = tf.placeholder(shape=(None), dtype=tf.float32, name='tf_x')
    tf_y = tf.placeholder(shape=(None), dtype=tf.float32, name='tf_y')
    
    #変数(モデルのパラメータ)を定義
    weight = tf.Variable(tf.random_normal(shape=(1,1), stddev=0.25), name='weight')
    bias = tf.Variable(0.0, name='bias')
    
    #モデルを構築
    #'+'も使用できるがtf.addは結果として得られるテンソルの名前をnameパラメータに指定できる
    y_hat = tf.add(weight*tf_x, bias, name='y_hat')
    
    #コストを計算
    cost = tf.reduce_mean(tf.square(tf_y - y_hat), name='cost')
    
    #モデルをトレーニング
    optim = tf.train.GradientDescentOptimizer(learning_rate=0.001)
    train_op = optim.minimize(cost, name='train_op')

In [3]:
#ランダムなデータセットを作成

np.random.seed(0)

def make_random_data():
    x = np.random.uniform(low=-2, high=4, size=200)
    y = []
    for t in x:
        r = np.random.normal(loc=0.0, scale=(0.5+t*t/3), size=None)
        y.append(r)
    return x, 1.726 * x -0.84 + np.array(y)

x, y = make_random_data()

In [4]:
x_train, y_train = x[:100], y[:100]
x_test, y_test = x[100:], y[100:]

n_epochs = 500

training_cost = []

with tf.Session(graph=g) as sess:
    #計算グラフにある変数をすべて初期化するための演算子を返す
    sess.run(tf.global_variables_initializer())
    print(sess.run(weight))
    
    for e in range(n_epochs):
        c, _ = sess.run([cost, train_op], feed_dict={tf_x: x_train, tf_y: y_train})
        training_cost.append(c)
        if not e % 50:
            print('Epoch %4d: %.4f' % (e, c))

[[0.08423644]]
Epoch    0: 9.1859
Epoch   50: 6.9381
Epoch  100: 5.8536
Epoch  150: 5.3063
Epoch  200: 5.0108
Epoch  250: 4.8362
Epoch  300: 4.7223
Epoch  350: 4.6408
Epoch  400: 4.5782
Epoch  450: 4.5278


## 【問題1】スクラッチを振り返る
ここまでのスクラッチを振り返り、ディープラーニングを実装するためにはどのようなものが必要だったかを列挙してください。


（例）

- 重みを初期化する必要があった
- エポックのループが必要だった

それらがフレームワークにおいてはどのように実装されるかを今回覚えていきましょう。

- 重み,バイアスの初期化方法を選択
    - SimpleInitializer(ガウス分布による初期化)
    - Xavier
    - HeInitializer(ReLUの時)
    
- 結合層の選択
    - 全結合
    - 畳み込み
    
- 活性化関数の選択
    - Sigmoid
    - Tanh
    - ReLU
    - Softmax(出力層)

- 最適化手法の選択
    - SGD(確率的勾配降下法)
    - AdaGrad
    
    
- 学習 
    - 重み, バイアスの初期化
    - forward
    - 誤差の計算
    - backward(パラメータの更新)
    - エポックのループ
- 推定

## データセットの用意
以前から使用しているIrisデータセットを使用します。以下のサンプルコードではIris.csvが同じ階層にある想定です。


Iris Species


目的変数はSpeciesですが、3種類ある中から以下の2種類のみを取り出して使用します。


- Iris-versicolor
- Iris-virginica

## 【問題2】スクラッチとTensorFlowの対応を考える
以下のサンプルコードを見て、先ほど列挙した「ディープラーニングを実装するために必要なもの」がTensorFlowではどう実装されているかを確認してください。


それを簡単に言葉でまとめてください。単純な一対一の対応であるとは限りません。


《サンプルコード》


＊バージョン1.5から1.14の間で動作を確認済みです。

In [5]:
"""
TensorFlowで実装したニューラルネットワークを使いIrisデータセットを2値分類する
"""
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
# データセットの読み込み
dataset_path ="Iris.csv"
df = pd.read_csv(dataset_path)
# データフレームから条件抽出
df = df[(df["Species"] == "Iris-versicolor")|(df["Species"] == "Iris-virginica")]
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = np.array(y)
X = np.array(X)
# ラベルを数値に変換
y[y=='Iris-versicolor'] = 0
y[y=='Iris-virginica'] = 1
y = y.astype(np.int)[:, np.newaxis]
# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [6]:
class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      訓練データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]

#### 値の設定  
ハイパーパラメータ -> 直接値を入力  
X, y -> tf.placeholder(データ型とshapeを指定)

In [7]:
# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1

# 計算グラフに渡す引数の形を決める
# tf.placeholderはデータが格納されている入れ物
# データは未定のままグラフを構築し、具体的な値は実行するときに与える
# 次元の大きさが可変の時,Noneを指定
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

#### 計算グラフの構築  
重みとバイアス -> tf.Valiable(学習時に更新する値)

それぞれの層を定義(結合方法, 活性化関数)

In [8]:
def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """
    # 重みとバイアスの宣言
    #tf.Valiableは変数(モデルのパラメータ,学習時に更新する値)を定義
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    
    #入力層,隠れ層
    #matmulは行列積,x@w+b
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    #ReLUによる活性化
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output


#### 目的関数, 最適化手法, 指標値


In [9]:
# ネットワーク構造の読み込み                               
logits = example_net(X)

# 目的関数
#reduce_meanは与えたリストに入っている数値の平均値を求める関数
#sigmoid_cross_entropy_with_logitsはシグモイド活性化関数でクロスエントロピーを取得する関数
#二値分類なので出力を0-1に収めるシグモイド関数
loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))

# 最適化手法
#Adam(魚本p175)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
#Adamでlossを最小化していく
train_op = optimizer.minimize(loss_op)

# 推定結果
#tf.equalは要素が一致するindexにTrueを返す
# tf.signは要素ごとに正なら1,0なら0,負なら-1となる変換をかける
#tf.sigmoid(logits)はy=1である確率, 50%を閾値にして-1,0,1に振り分け
correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))

# 指標値計算
#tf.castでbool型からfloat32に変換
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


#### 計算グラフの実行

In [10]:
# variableの初期化
#計算グラフにある変数をすべて初期化するための演算子を返す
init = tf.global_variables_initializer()

# 計算グラフの実行
#Sessionで計算グラフを起動し、計算グラフの様々なノードを実行する

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        #np.ceilは小数点の切り上げ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            #値はSession.run()の引数feed_dictに辞書型で指定する
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= total_batch
        total_acc /= total_batch
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
#         print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
        print("Epoch {}, total_loss : {:.4f}, val_loss : {:.4f}, total_acc : {:.3f}, val_acc : {:.3f}".format(epoch, total_loss, val_loss, total_acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

Epoch 0, total_loss : 25.8372, val_loss : 12.5255, total_acc : 0.664, val_acc : 0.625
Epoch 1, total_loss : 13.3242, val_loss : 0.0056, total_acc : 0.700, val_acc : 1.000
Epoch 2, total_loss : 1.7133, val_loss : 0.0191, total_acc : 0.857, val_acc : 1.000
Epoch 3, total_loss : 0.3963, val_loss : 0.5059, total_acc : 0.971, val_acc : 0.938
Epoch 4, total_loss : 0.3000, val_loss : 0.0004, total_acc : 0.943, val_acc : 1.000
Epoch 5, total_loss : 0.2845, val_loss : 0.5157, total_acc : 0.986, val_acc : 0.938
Epoch 6, total_loss : 0.2547, val_loss : 0.0023, total_acc : 0.957, val_acc : 1.000
Epoch 7, total_loss : 0.2589, val_loss : 0.8435, total_acc : 0.986, val_acc : 0.875
Epoch 8, total_loss : 0.5799, val_loss : 1.5683, total_acc : 0.943, val_acc : 0.812
Epoch 9, total_loss : 0.9330, val_loss : 5.8528, total_acc : 0.921, val_acc : 0.812
test_acc : 0.750


# 他のデータセットへの適用

これまで扱ってきた小さなデータセットがいくつかあります。上記サンプルコードを書き換え、これらに対して学習・推定を行うニューラルネットワークを作成してください。


- Iris（3種類全ての目的変数を使用）
- House Prices

どのデータセットもtrain, val, testの3種類に分けて使用してください。

## 【問題3】3種類全ての目的変数を使用したIrisのモデルを作成
Irisデータセットのtrain.csvの中で、目的変数Speciesに含まれる3種類全てを分類できるモデルを作成してください。


Iris Species


2クラスの分類と3クラス以上の分類の違いを考慮してください。それがTensorFlowでどのように書き換えられるかを公式ドキュメントなどを参考に調べてください。


《ヒント》


以下の2箇所は2クラス分類特有の処理です。

In [11]:
loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))

In [12]:
correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))

メソッドは以下のように公式ドキュメントを確認してください。


tf.nn.sigmoid_cross_entropy_with_logits  |  TensorFlow


tf.math.sign  |  TensorFlow


＊tf.signとtf.math.signは同じ働きをします。

#### 変えること  
目的変数のonehot化  
n_class=3  
目的関数 sigmoid -> softmax  
推定結果tf.sign -> tf.argmax

In [13]:
from sklearn.preprocessing import OneHotEncoder

In [14]:
# データセットの読み込み
dataset_path ="Iris.csv"
df = pd.read_csv(dataset_path)

y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = np.array(y)
X = np.array(X)

# ワンホットライブラリのインスタンス作成
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y = enc.fit_transform(y[:, np.newaxis])

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [15]:
# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 3

# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

In [16]:
def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output


In [17]:
# ネットワーク構造の読み込み                               
logits = example_net(X)

# 目的関数(tf.nn.sigmoid_cross_entropy_with_logitsを変更)
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=logits))

# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# 推定結果(tf.sign->tf.argmax, tf.sigmoid->tf.nn.softmax)
#tf.argmaxの第2引数を1にして行ごとの最大を返す
correct_pred = tf.equal(tf.argmax(Y,1),tf.argmax(tf.nn.softmax(logits),1))

# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [18]:
# variableの初期化
init = tf.global_variables_initializer()

# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        #ミニバッチの総数
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= total_batch
        total_acc /= total_batch
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
#         print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
        print("Epoch {}, total_loss : {:.4f}, val_loss : {:.4f}, total_acc : {:.3f}, val_acc : {:.3f}".format(epoch, total_loss, val_loss, total_acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

Epoch 0, total_loss : 11.1145, val_loss : 7.8915, total_acc : 0.643, val_acc : 0.708
Epoch 1, total_loss : 2.4986, val_loss : 11.5099, total_acc : 0.793, val_acc : 0.625
Epoch 2, total_loss : 2.0065, val_loss : 1.1870, total_acc : 0.850, val_acc : 0.917
Epoch 3, total_loss : 0.4586, val_loss : 2.6469, total_acc : 0.940, val_acc : 0.708
Epoch 4, total_loss : 0.8553, val_loss : 3.0087, total_acc : 0.900, val_acc : 0.750
Epoch 5, total_loss : 1.1043, val_loss : 6.4418, total_acc : 0.900, val_acc : 0.625
Epoch 6, total_loss : 1.7096, val_loss : 1.4422, total_acc : 0.890, val_acc : 0.833
Epoch 7, total_loss : 0.2102, val_loss : 5.9209, total_acc : 0.970, val_acc : 0.667
Epoch 8, total_loss : 0.6045, val_loss : 10.9283, total_acc : 0.930, val_acc : 0.625
Epoch 9, total_loss : 2.3174, val_loss : 18.3258, total_acc : 0.843, val_acc : 0.625
test_acc : 0.833


## 【問題4】House Pricesのモデルを作成
回帰問題のデータセットであるHouse Pricesを使用したモデルを作成してください。


House Prices: Advanced Regression Techniques


この中のtrain.csvをダウンロードし、目的変数としてSalePrice、説明変数として、GrLivAreaとYearBuiltを使ってください。説明変数はさらに増やしても構いません。


分類問題と回帰問題の違いを考慮してください。

#### 変えること

標準化  
学習率  
目的変数は1列  
目的関数を平均2乗誤差(MSE)  
指標値計算なし  

In [56]:
from sklearn.preprocessing import StandardScaler

In [57]:
# データセットの読み込み
dataset_path ="train.csv"
df = pd.read_csv(dataset_path)

y = df["SalePrice"]
X = df.loc[:, ["GrLivArea", "YearBuilt"]]
y = np.array(y)
X = np.array(X)

print(y.shape)
y = y[:, np.newaxis]
print(y.shape)

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

#標準化
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)

(1460,)
(1460, 1)


In [58]:
# ハイパーパラメータの設定
learning_rate = 0.00001
batch_size = 10
num_epochs = 100
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]

# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, 1])

# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

In [59]:
def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output

In [60]:
# ネットワーク構造の読み込み                               
logits = example_net(X)

# 目的関数
#tf.squareで要素ごとの2乗をとり、tf.reduce_meanで平均
loss_op = tf.reduce_mean(tf.square(Y - logits))

# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

In [61]:
# variableの初期化
init = tf.global_variables_initializer()

# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss = sess.run(loss_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
        total_loss /= total_batch
        val_loss = sess.run(loss_op, feed_dict={X: X_val, Y: y_val})
#         print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}".format(epoch, loss, val_loss))
        print("Epoch {}, total_loss : {:.4f}, val_loss : {:.4f}".format(epoch, total_loss, val_loss))
   

Epoch 0, total_loss : 39323830468.0851, val_loss : 38320234496.0000
Epoch 1, total_loss : 39323635973.4468, val_loss : 38320041984.0000
Epoch 2, total_loss : 39323441653.1064, val_loss : 38319853568.0000
Epoch 3, total_loss : 39323246483.0638, val_loss : 38319665152.0000
Epoch 4, total_loss : 39323050920.8511, val_loss : 38319464448.0000
Epoch 5, total_loss : 39322854269.2766, val_loss : 38319267840.0000
Epoch 6, total_loss : 39322659273.5319, val_loss : 38319075328.0000
Epoch 7, total_loss : 39322462142.6383, val_loss : 38318870528.0000
Epoch 8, total_loss : 39322264881.0213, val_loss : 38318669824.0000
Epoch 9, total_loss : 39322067946.2128, val_loss : 38318473216.0000
Epoch 10, total_loss : 39321870553.8723, val_loss : 38318280704.0000
Epoch 11, total_loss : 39321672878.2979, val_loss : 38318080000.0000
Epoch 12, total_loss : 39321473590.4681, val_loss : 38317883392.0000
Epoch 13, total_loss : 39321275239.4894, val_loss : 38317686784.0000
Epoch 14, total_loss : 39321076365.6170, val

## 【問題5】MNISTのモデルを作成

ニューラルネットワークのスクラッチで使用したMNISTを分類するモデルを作成してください。


3クラス以上の分類という点ではひとつ前のIrisと同様です。入力が画像であるという点で異なります。


スクラッチで実装したモデルの再現を目指してください。

#### 変えること  

平坦化  
正規化(データを0-1の範囲に押し込める)  
->画像は値の範囲が決まってるため(ピクセルのRGB値0-255)正規化が用いられることが多い  
目的変数のonehot化  
n_class=10  
目的関数 sigmoid -> softmax  
推定結果tf.sign -> tf.argmax  

In [62]:
from keras.datasets import mnist

In [63]:
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder

In [64]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# 平坦化
X_train = X_train.reshape(-1, 784)
X_test = X_test.reshape(-1, 784)

# 前処理
X_train = X_train.astype(np.float)
X_test = X_test.astype(np.float)

#正規化
X_train /= 255
X_test /= 255

# ワンホットライブラリのインスタンス作成
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_train = enc.fit_transform(y_train[:, np.newaxis])
y_test = enc.fit_transform(y_test[:, np.newaxis])

# trainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)


In [65]:
# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 10

# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

In [66]:
def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output

In [67]:
# ネットワーク構造の読み込み                               
logits = example_net(X)

# 目的関数
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=logits))

# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# 推定結果
correct_pred = tf.equal(tf.argmax(Y,1),tf.argmax(tf.nn.softmax(logits),1))

# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [68]:
# variableの初期化
init = tf.global_variables_initializer()

# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= total_batch
        total_acc /= total_batch
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
#         print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
        print("Epoch {}, total_loss : {:.4f}, val_loss : {:.4f}, total_acc : {:.3f}, val_acc : {:.3f}".format(epoch, total_loss, val_loss, total_acc, val_acc))
   
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

Epoch 0, total_loss : 4.9291, val_loss : 1.1050, total_acc : 0.766, val_acc : 0.757
Epoch 1, total_loss : 0.6998, val_loss : 0.6621, total_acc : 0.827, val_acc : 0.854
Epoch 2, total_loss : 0.4213, val_loss : 0.3224, total_acc : 0.895, val_acc : 0.913
Epoch 3, total_loss : 0.2946, val_loss : 0.3181, total_acc : 0.925, val_acc : 0.929
Epoch 4, total_loss : 0.2575, val_loss : 0.2888, total_acc : 0.934, val_acc : 0.933
Epoch 5, total_loss : 0.2405, val_loss : 0.2927, total_acc : 0.939, val_acc : 0.933
Epoch 6, total_loss : 0.2378, val_loss : 0.2834, total_acc : 0.942, val_acc : 0.935
Epoch 7, total_loss : 0.2343, val_loss : 0.2751, total_acc : 0.942, val_acc : 0.934
Epoch 8, total_loss : 0.2253, val_loss : 0.2991, total_acc : 0.945, val_acc : 0.939
Epoch 9, total_loss : 0.2208, val_loss : 0.2601, total_acc : 0.945, val_acc : 0.935
test_acc : 0.938


# TensorFlow入門2 ロジスティック回帰実装

## ロジスティック回帰の実装

TensorFlowを使いロジスティック回帰を実装していきます。入門1では単純な足し算でしたが、ここでは学習を伴う計算を行います。

In [45]:
import numpy as np
x_train = np.array([[0,0],[0,1],[1,0],[1,1]])
y_train = np.array([[0],[0],[0],[1]])

In [46]:
import tensorflow as tf
x = tf.placeholder(tf.float32, [None, 2])
t = tf.placeholder(tf.float32, [None, 1])

In [47]:
W = tf.Variable(tf.zeros([2,1]))
b = tf.Variable(tf.zeros([1]))

In [48]:
# y = tf.sigmoid(tf.matmul(x, W) + b)

mat = tf.matmul(x, W)
y = tf.sigmoid(mat + b)

cross_entropy = tf.reduce_sum(-t * tf.log(y) - (1 - t) * tf.log(1 - y))

In [49]:
train_step = tf.train.GradientDescentOptimizer(0.1).minimize(cross_entropy)

In [50]:
correct_prediction = tf.equal(tf.sign(y - 0.5), tf.sign(t - 0.5))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [51]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [52]:
for epoch in range(1000):
    sess.run(train_step, feed_dict={
        x:x_train,
        t:y_train
    })
# 100回ごとに正解率を表示
    if epoch % 100 == 0:
        acc_val = sess.run(
            accuracy, feed_dict={
                x:x_train,
                t:y_train})
        print ('epoch: %d, Accuracy: %f'
               %(epoch, acc_val))
        print(sess.run(mat, feed_dict={x:x_train,t:y_train}))

epoch: 0, Accuracy: 0.750000
[[0.]
 [0.]
 [0.]
 [0.]]
epoch: 100, Accuracy: 1.000000
[[0.       ]
 [1.7671354]
 [1.7671354]
 [3.5342708]]
epoch: 200, Accuracy: 1.000000
[[0.       ]
 [2.7020476]
 [2.7020476]
 [5.404095 ]]
epoch: 300, Accuracy: 1.000000
[[0.       ]
 [3.3457706]
 [3.3457706]
 [6.691541 ]]
epoch: 400, Accuracy: 1.000000
[[0.       ]
 [3.8412282]
 [3.8412282]
 [7.6824565]]
epoch: 500, Accuracy: 1.000000
[[0.       ]
 [4.2443557]
 [4.2443557]
 [8.488711 ]]
epoch: 600, Accuracy: 1.000000
[[0.       ]
 [4.5839767]
 [4.5839767]
 [9.1679535]]
epoch: 700, Accuracy: 1.000000
[[0.       ]
 [4.8771544]
 [4.8771544]
 [9.754309 ]]
epoch: 800, Accuracy: 1.000000
[[ 0.       ]
 [ 5.1348853]
 [ 5.1348853]
 [10.269771 ]]
epoch: 900, Accuracy: 1.000000
[[ 0.       ]
 [ 5.3646903]
 [ 5.3646903]
 [10.729381 ]]


In [53]:
#学習結果が正しいか確認
classified = sess.run(correct_prediction, feed_dict={
    x:x_train,
    t:y_train
})
#出力yの確認
prob = sess.run(y, feed_dict={
    x:x_train,
    t:y_train
})
print(classified)
# [[ True]
# [ True]
# [ True]
# [ True]]
print(prob)
# [[  1.96514215e-04]
# [  4.90498319e-02]
# [  4.90498319e-02]
# [  9.31203783e-01]]

[[ True]
 [ True]
 [ True]
 [ True]]
[[1.9654632e-04]
 [4.9049824e-02]
 [4.9049824e-02]
 [9.3120384e-01]]


In [54]:
print('W:', sess.run(W))
print('b:', sess.run(b))
# W: [[ 5.5699544]
# [ 5.5699544]]
# b: [-8.53457928]

W: [[5.5699544]
 [5.5699544]]
b: [-8.534579]


In [55]:
sess.close()